Extract data from https://www.climateimpact.com

In [1]:
import json
import pandas as pd
with open('../input/ClimatePartnerImpact/projects_data.json', 'r') as file:
    data = json.load(file)

print(type(data))


print(data[0])


if isinstance(data, list) and isinstance(data[0], dict):
    cp_df = pd.DataFrame(data)
    cp_df.info()

<class 'list'>
{'id': 'omper396lw3gxwq0', 'type': 'Feature', 'geometry_type': 'Point', 'geometry_coordinates': [[25.2555, -32.8334]], 'properties_id': 'omper396lw3gxwq0', 'properties_name': 'Somerset East Spekboom Restoration', 'properties_slug': 'cpi2301', 'properties_type': 'project', 'properties_image': 'YFs3KVkT5wyLQp2XALcDPjv9MuhpUa4sPmlQbUXo.jpg', 'properties_region': 'Eastern Cape', 'properties_countries': [['South Africa']], 'properties_activity_type': [['Restoration']]}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        7 non-null      object
 1   type                      7 non-null      object
 2   geometry_type             7 non-null      object
 3   geometry_coordinates      7 non-null      object
 4   properties_id             7 non-null      object
 5   properties_name           7 non-

In [2]:
cp_df.head()

,id,type,geometry_type,geometry_coordinates,properties_id,properties_name,properties_slug,properties_type,properties_image,properties_region,properties_countries,properties_activity_type
0,omper396lw3gxwq0,Feature,Point,"[[25.2555, -32.8334]]",omper396lw3gxwq0,Somerset East Spekboom Restoration,cpi2301,project,YFs3KVkT5wyLQp2XALcDPjv9MuhpUa4sPmlQbUXo.jpg,Eastern Cape,[[South Africa]],[[Restoration]]
1,g0drw38lmmjx4z9q,Feature,Point,"[[29.6963, -2.1423]]",g0drw38lmmjx4z9q,Rwanda Bamboo River Restoration,cpi2207,project,sCEOaHMQkFtuiXjR7Q2Ke8puX9pPrJCyqLorQmq0.jpg,"Muhanga, Ruhango & Karongi",[[Rwanda]],[[Restoration]]
2,ln5pa3mml739ogex,Feature,Point,"[[26.3263, -33.2508]]",ln5pa3mml739ogex,Mahkanda Spekboom Restoration,cpi2203,project,icVp9uj48DKUSjliX2dPQUmKTUDPVz9JyYRVIYYf.jpg,Eastern Cape,[[South Africa]],[[Restoration]]
3,wermv3llmd36koql,Feature,Point,"[[85.5019, 23.0209]]",wermv3llmd36koql,East India Tasar Afforestation,cpi2209,project,C3WduX0jsTP3xILKpdH2ZGfGdhYlilIgWV6iEJHg.jpg,Simdega & Purulia,[[India]],[[Plantation Forestry]]
4,q4d7myr9gkya6kzx,Feature,Point,"[[-0.8182, 6.8247]]",q4d7myr9gkya6kzx,North Bandai Bamboo Reforestation,cpi2103,project,XTk0EmUQhxHvtQOjfUHiXel0W4CtLjchNVfiz4zH.jpg,Ashanti,[[Ghana]],[[Plantation Forestry]]


In [3]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'id': 'site_id_reported',
    'properties_id': 'project_id_reported',
    'geometry_coordinates': 'geometry_reported',
    # 'data.forests.short_desc': 'description_reported',
    # 'country': 'country',
    # 'data.forests.main_image': 'url',
    #'survivalRate': 'survival_rate_reported',
}

df = cp_df[list(columns_rename_mapping.keys())]


df.rename(columns=columns_rename_mapping, inplace=True)

/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_3723/1474449437.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=columns_rename_mapping, inplace=True)


In [5]:
#df['site_id_reported'] = ['tist_asia_{}'.format(i) for i in range(1, len(asia_df) + 1)]
df = df.assign(description_reported=None,site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,country=None,site_id_reported=None,)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   site_id_reported          0 non-null      object
 1   project_id_reported       7 non-null      object
 2   geometry_reported         7 non-null      object
 3   description_reported      0 non-null      object
 4   site_sqkm                 0 non-null      object
 5   species_count_reported    0 non-null      object
 6   species_planted_reported  0 non-null      object
 7   survival_rate_reported    0 non-null      object
 8   trees_planted_reported    0 non-null      object
 9   planting_date_reported    0 non-null      object
 10  country                   0 non-null      object
dtypes: object(11)
memory usage: 744.0+ bytes


In [ ]:
df.to_csv('../input/ClimatePartnerImpact/projects_data.csv', index=False)